In [ ]:
#@markdown  <font color='green'><font size=5>**1. Setup and Connect to Drive**</font>
!pip install gspread_formatting
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
import cv2
import yaml
import sys
import os
from IPython.display import display, HTML, clear_output
import time
import threading
import torch
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from gspread_formatting import *
from google.colab import drive

def funcy(worksheet, df):
  # Get the column names and values from the DataFrame
  header = df.columns.values.tolist()
  values = df.values.tolist()

  header.insert(0, 'image\split')
  for i, row in enumerate(values):
      row.insert(0, df.index[i])

  worksheet.update([header] + values)

  set_column_width(worksheet, 'A', 180)
  set_column_width(worksheet, 'B:Q', 80)

  worksheet.format(f"A1", {
      "horizontalAlignment": "CENTER",
      "textFormat": {
        "bold": False
      }
  })
  worksheet.format("A:Z", {
      # "horizontalAlignment": "CENTER",
      "textFormat": {
        "fontSize": 9,
        "bold": False
      }
  })
  rule = ConditionalFormatRule(
      ranges=[GridRange.from_a1_range('B1:Q', worksheet)],
      booleanRule=BooleanRule(
          condition=BooleanCondition('TEXT_CONTAINS', ['(']),
          format=CellFormat(textFormat=textFormat(foregroundColor=Color(1,0,0), bold=False))#, backgroundColor=Color(1,0,0))
      )
  )
  rule2 = ConditionalFormatRule(
      ranges=[GridRange.from_a1_range(f'B2:Q{len(df.index)+1}', worksheet)],
      booleanRule=BooleanRule(
          condition=BooleanCondition('TEXT_NOT_CONTAINS', ['(']),
          format=CellFormat(textFormat=textFormat(foregroundColor=Color(0,1,0), bold=False))#, backgroundColor=Color(0,1,0))
      )
  )

  rules = get_conditional_format_rules(worksheet)
  rules.clear()
  rules.append(rule)
  rules.append(rule2)
  rules.save()
  clear_output()

def create_df():
  for i,f_path in enumerate(folder_paths):
    images = os.listdir(f_path)
    df = pd.DataFrame(columns=range(16), index=images)
    for image_ in df.index:
      img_path = f'{f_path}/{image_}'
      lll = []
      for s in sorted([_ for _ in os.listdir(img_path) if _.startswith("split_")], key=lambda x: int(x.split("_")[-1])):
        sp_ = int(s.split("_")[-1])
        split_path = f'{img_path}/{s}'
        files_in_folder = os.listdir(split_path)
        with open(f'{split_path}/seg_{sp_}.yaml') as f:
              seg = yaml.safe_load(f)
        if f'anns_{sp_}.yaml' in files_in_folder:
            with open(f'{split_path}/anns_{sp_}.yaml') as f:
              anns = yaml.safe_load(f)
            annotated_masks = [x for x in list(anns[image_][sp_].keys()) if anns[image_][sp_][x]!=""]
            not_annoted_masks = [m for m in seg[image_][sp_] if m not in annotated_masks]
            masks_list = not_annoted_masks+annotated_masks
            n = len(masks_list)
            annted = len(annotated_masks)
            if annted!=n:
              lll.append(f"{annted}/{n} ({n-annted})")
            else:
              lll.append(f"{annted}/{n}")
        else:
          l =len(seg[image_][sp_])
          lll.append(f"0/{l} ({l})")
      df.loc[image_] = lll
    worksheet = sh.get_worksheet(i)
    funcy(worksheet, df)

def show_image_and_split(image,split):
  im_path = f'{folder_path}/{image}'
  split_path = f'{im_path}/split_{split}'
  image_path = f'{im_path}/{image}.jpg'
  img = cv2.imread(f'{im_path}/{image}.jpg')
  cv2.rectangle(img, ((split%4)*(img.shape[1]//4), (split//4)*(img.shape[0]//4)),
                    ((1+split%4)*(img.shape[1]//4), (1+split//4)*(img.shape[0]//4)),
                    color=(255, 0, 0), thickness=30)
  img2 = cv2.imread(f'{split_path}/split_{split}.jpg')
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
  if torch.cuda.is_available():
    files_in_folder = os.listdir(split_path)
    if f'seg_{split}.yaml' in files_in_folder:
      with open(f'{split_path}/seg_{split}.yaml') as f:
        seg = yaml.safe_load(f)
      masked_image_path = f'{split_path}/masked_image.jpg'
      msks = [seg[image][split][mask] for mask in seg[image][split]]
      if not 'masked_image.jpg' in files_in_folder:
        plot_anns(img2,msks, masked_image_path, image, split)
      img3= cv2.imread(masked_image_path)
      img3 = cv2.cvtColor(img3, cv2.COLOR_BGR2RGB)
      # cv2_imshow(img3)
      fig, ax = plt.subplots(1, 3, figsize=(15, 5))
      ax[0].imshow(img)
      ax[0].set_title(f'image {image}', color="r")
      ax[1].imshow(img2)
      ax[1].set_title(f'split {split}', color="r")
      ax[2].imshow(img3)
      ax[2].set_title(f'{len(msks)} masks', color="r")

      for axis in ax:
          axis.axis('off')
      plt.tight_layout()
      plt.show()
      #plt.clf()
  else:
    fig, ax = plt.subplots(1, 2, figsize=(10, 5))
    ax[0].imshow(img)
    ax[0].set_title(f'image {image}', color="r")
    ax[1].imshow(img2)
    ax[1].set_title(f'split {split}', color="r")
    for axis in ax:
        axis.axis('off')
    plt.tight_layout()
    plt.show()

def show_anns(anns):
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)
    polygons = []
    color = []
    for ann in sorted_anns:
        m = rle_to_mask(ann['segmentation'])
        img = np.ones((m.shape[0], m.shape[1], 3))
        color_mask = np.random.random((1, 3)).tolist()[0]
        for i in range(3):
            img[:,:,i] = color_mask[i]
        ax.imshow(np.dstack((img, m*0.35)))

def print_categories():
  grouped_categories = [['Metal (general)', 'Metal (iron bender)', 'Metal (pipe)', 'Wood (pallett)', 'Wood (scraps/cuttings)','Carton','Nylon'],
      ['Plastic (big bag)', 'Plastic (bucket)', 'Plastic (general)', 'Plastic (pipe)', 'Plastic (sand bag)', 'Styrofoam','Paper'],
      ['Textil','Gypsum','Concrete', 'Ceramics','Rubber','Background', 'All Background'],
      ['Unknown','Skip','Back','Exit','Zoom']]

  num_columns = len(grouped_categories)
  max_group_size = max(len(group) for group in grouped_categories)
  rows = []
  for i in range(max_group_size):
      row = []
      for j in range(num_columns):
          category_group = grouped_categories[j]
          if i < len(category_group):
              row.append(category_group[i])
          else:
              row.append('')
      rows.append(row)
  column_width = max(len(category) for row in rows for category in row) + 2
  print()
  for row in rows:
    formatted_row=[]
    for category in row:
      if category in categories:
        if categories.index(category)<=9:
          f = f'{categories.index(category)}:  \033[1;32m{category:<{column_width}}\033[0m'
        else:
          f = f'{categories.index(category)}: \033[1;32m{category:<{column_width}}\033[0m'
        formatted_row.append(f)
      else:
        formatted_row.append(f'{category:<{column_width}}')
    print(''.join(formatted_row))

def show_masks_grid(image_name, sp, select = "All"):
  split_path = f'{folder_path}/{image_name}/split_{sp}'
  listdir = os.listdir(split_path)
  if f'anns_{sp}.yaml' in listdir:
    with open(f'{split_path}/anns_{sp}.yaml', 'r') as f:
        ann_dict = yaml.safe_load(f)
  else:
    ann_dict={image_name:{sp:{}}}
  for k in listdir:
    if k.startswith("mask_"):
      if k.split(".")[0] not in ann_dict[image_name][sp]:
        ann_dict[image_name][sp][k.split(".")[0]]=""
  with open(f'{split_path}/anns_{sp}.yaml', 'w') as f:
    yaml.dump(ann_dict,f)
  images_ = []
  for k in listdir:
    if k.startswith("mask_"):
      if (ann_dict[image_name][sp][k.split(".")[0]]=="" and select in ["All","Not Annotated"])\
        or (ann_dict[image_name][sp][k.split(".")[0]]!="" and select in ["All","Annotated"]):
        images_.append(f'{split_path}/{k}')


  n = len(images_)
  if n==0: return
  x= f"{select}: {n}"
  output = f"<p style='font-size: 18px; '>{x}</p>"
  display(HTML(output))
  cols = min(n,10)
  rows = (n//cols)+int(n%cols!=0)
  fig, axs = plt.subplots(rows, cols, figsize=(cols*1.5, rows*1.5))
  plt.subplots_adjust(wspace=0.5, hspace=0.5)
  for i in range(rows):
    for j in range(cols):
      if rows > 1:
        z = axs[i, j]
      elif cols>1:
        z = axs[j]
      else:
        z=axs
      z.axis('off')
  for idx, image_path in enumerate(images_):

      row = idx // cols
      col = idx % cols
      if rows > 1:
        ax = axs[row, col]
      elif cols>1:
        ax = axs[col]
      else:
        ax=axs
      img = plt.imread(image_path)
      ax.imshow(img)
      ax.axis('off')
      mask_name = image_path.split("/")[-1].split(".")[0]
      ann = ""
      if mask_name in ann_dict[image_name][sp]:
        ann = f" {ann_dict[image_name][sp][mask_name]}"

      m_num = f'{image_path.split("/")[-1].split(".")[0].split("_")[-1]}{ann}'
      ax.set_title(f'{m_num}', fontsize=8, color="r")

  plt.show()

def get_box_from_mask(mask):
  mask = rle_to_mask(mask)
  pixel_list = np.argwhere(mask)
  min_x = min(pixel[1] for pixel in pixel_list)
  min_y = min(pixel[0] for pixel in pixel_list)
  max_x = max(pixel[1] for pixel in pixel_list)
  max_y = max(pixel[0] for pixel in pixel_list)
  box = [min_x, min_y, max_x, max_y]
  return box

def plot_anns(image, masks, path, image_name, split):
  plt.imshow(image)
  show_anns(masks)
  plt.axis('off')
  #plt.title(f"{image_name} / split {split}\n({len(masks)} masks)", color = "r")
  plt.savefig(path)

def annotate_one_sp(image_name, sp, specific_masks=[], small_images = True):
  start = time.time()
  im_path = f'{folder_path}/{image_name}'
  split_path = f'{im_path}/split_{sp}'
  with open(f'{split_path}/seg_{sp}.yaml') as f:
      seg = yaml.safe_load(f)
  with open(f'{split_path}/box_{sp}.yaml', 'r') as f:
      boxs = yaml.safe_load(f)
  files_in_folder = os.listdir(split_path)
  ann_dict={image_name:{sp:{}}}
  if f'anns_{sp}.yaml' in files_in_folder:
    with open(f'{split_path}/anns_{sp}.yaml', 'r') as f:
        ann_dict = yaml.safe_load(f)
  else:
    ann_dict={image_name:{sp:{}}}
  image = cv2.imread(f'{split_path}/split_{sp}.jpg')
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  if specific_masks!=[]:
    annotated_masks = [x for x in list(ann_dict[image_name][sp].keys()) if int(x.split('_')[-1]) in specific_masks]
    not_annoted_masks = [m for m in seg[image_name][sp] if m not in annotated_masks and int(m.split('_')[-1]) in specific_masks]
  else:
    annotated_masks = [x for x in list(ann_dict[image_name][sp].keys()) if ann_dict[image_name][sp][x]!=""]
    not_annoted_masks = [m for m in seg[image_name][sp] if m not in annotated_masks]
  masks_list = not_annoted_masks+annotated_masks
  n = len(masks_list)
  annted = len(annotated_masks)
  m = 0
  anns_made = 0
  zoom=False
  while m < n and m>=0:
    plt.gcf()
    plt.clf()
    clear_output()
    print(f"\033[34m{'total: '}\033[0m{n}\n\033[34m{'annotated: '}\033[0m{annted+anns_made}\n\033[34m{'not annotated: '}\033[0m{n-annted-anns_made}\n")
    if not f'{masks_list[m]}.jpg' in files_in_folder: continue
    print(f"\033[1;36m{'**************** mask '}{m+1}{' of '}{n}{' ****************'}\033[0m")
    mask = seg[image_name][sp][masks_list[m]]
    box = boxs[image_name][sp][masks_list[m]]
    cropped_image = cv2.imread(f'{split_path}/{masks_list[m]}.jpg')
    imagec = image.copy()
    cv2.rectangle(imagec, (box[0],box[1]),(box[2],box[3]), color=(255, 0, 0), thickness=4)
    cropped_image = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB)
    fig, ax = plt.subplots(1, 2, figsize=(10, 5))
    ax[0].imshow(cropped_image)
    ax[0].set_title(f'split {sp}', color="r")
    ax[1].imshow(imagec)
    ax[1].set_title(f'mask {masks_list[m]}', color="r")
    for axis in ax:
        axis.axis('off')
    plt.tight_layout()
    plt.show()
    #cv2_imshow(cropped_image)
    if zoom:
      imagec = cv2.cvtColor(imagec, cv2.COLOR_BGR2RGB)
      cv2_imshow(imagec)
      zoom=False
    if masks_list[m] in ann_dict[image_name][sp] and ann_dict[image_name][sp][masks_list[m]]!="" and specific_masks==[]:
      print(f"\nAnnotated: \033[1;32m{ann_dict[image_name][sp][masks_list[m]]}\033[0m. do you want to skip? [y\\n]")
      con = input()
      if con == 'y':
        m+=1
        continue
      elif (con.isdigit() and int(con) == categories.index('Back')) or con == 'back':
        m-=1
        continue
      elif (con.isdigit() and int(con) == categories.index('Exit')) or con == 'exit':
        if not ann_dict[image_name][sp]=={}:
          with open(f'{split_path}/anns_{sp}.yaml', 'w') as f:
            yaml.dump(ann_dict,f)
        break
    print_categories()
    try:
        ans = input("\nenter category number:\n")
    except:
        break
    if ans=="b":
      ans=str(categories.index('Background'))
    if ans == "u":
      ans =str(categories.index('Unknown'))
    if (ans.isdigit() and int(ans) == categories.index('Exit')) or ans =="exit":
      if not ann_dict[image_name][sp]=={}:
        with open(f'{split_path}/anns_{sp}.yaml', 'w') as f:
          yaml.dump(ann_dict,f)
      break
    elif ans.isdigit() and int(ans) == categories.index('Zoom'):
      zoom=True
      continue
    elif ans.isdigit() and (int(ans) == categories.index('Skip')\
          or int(ans)>=len(categories) or int(ans)<0):
      m+=1
      continue
    elif ans.isdigit() and int(ans) == categories.index('Back'):
      m = max(0, m-1)
    elif ans.isdigit() and int(ans) == categories.index('All Background'):
      for ma in masks_list:
        ann_dict[image_name][sp][ma] = 'Background'
      with open(f'{split_path}/anns_{sp}.yaml', 'w') as f:
          yaml.dump(ann_dict,f)
      anns_made = n
      break
    elif ans.isdigit() and int(ans) in range(len(categories)) :
      selected_category = categories[int(ans)]
      ann_dict[image_name][sp][masks_list[m]] = selected_category
      with open(f'{split_path}/anns_{sp}.yaml', 'w') as f:
          yaml.dump(ann_dict,f)
      print(f"\033[1;95m{selected_category}\033[0m")
      m+=1
      anns_made += 1
    else:
      m+=1
  end = time.time()
  clear_output()
  if n-annted-anns_made!=0:
    df.loc[image_name][sp] = f"{annted+anns_made}/{n} ({n-annted-anns_made})"
  else:
    df.loc[image_name][sp] = f"{annted+anns_made}/{n}"
  funcy(worksheet, df)
  cc= f"{anns_made}{' annotation in '}{(end-start)/60:.2f}{' minuts.'}"
  output = f"<p style='font-size: 20px; color: 	Maroon;'>{cc}</p>"
  display(HTML(output))


if torch.cuda.is_available():
  import torchvision
  print("PyTorch version:", torch.__version__)
  print("Torchvision version:", torchvision.__version__)
  print("CUDA is available:", torch.cuda.is_available())
  import sys
  !{sys.executable} -m pip install opencv-python matplotlib
  !{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'
  !wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

  from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor
  from segment_anything.utils.amg import *
  from pycocotools import mask as mask_utils


drive.mount('/content/drive')
creds = ServiceAccountCredentials.from_json_keyfile_name('/content/drive/MyDrive/Project/service_account.json')
gc = gspread.authorize(creds)
# spreadsheet = gc.open("status")
sh = gc.open("status")
categories =  [
              'Metal (general)',
              'Metal (iron bender)',
              'Metal (pipe)',
              'Wood (pallett)',
              'Wood (scraps/cuttings)',
              'Carton',
              'Nylon',
              'Plastic (big bag)',
              'Plastic (bucket)',
              'Plastic (general)',
              'Plastic (pipe)',
              'Plastic (sand bag)',
              'Styrofoam',
              'Paper',
              'Textil',
              'Gypsum',
              'Concrete',
              'Ceramics',
              'Rubber',
              'Background',
              'All Background',
              'Unknown',
              'Skip',
              'Back',
              'Exit',
              'Zoom'
              ]

folder_paths = ["/content/drive/MyDrive/Project/data/Adi","/content/drive/MyDrive/Project/data/Yoni","/content/drive/MyDrive/Project/data/Arik"]

clear_output()



In [ ]:
#@markdown <font color='green'><font size=5>**2.  Choose Image and Split**</font>

#### Adi ####
# iii = 0
# image = '20230215_093545' #@param ['DJI_20230221095630_0096_V','DJI_20230221095511_0068_V','DJI_20230221095408_0051_V','DJI_20230221095224_0017_V','DJI_20230221095221_0015_V','DJI_20230221095237_0023_V','DJI_20230221095259_0031_V','DJI_20230221095102_0059_V','DJI_20230221095159_0005_V','DJI_20230221095210_0009_V','DJI_20230221095002_0013_V','DJI_20230221095039_0041_V','DJI_20230221094015_0129_V','DJI_20230221091844_0111_V','DJI_20230221094002_0116_V','DJI_20230221095048_0048_V','DJI_20230221091303_0087_V','DJI_20230221091321_0094_V','DJI_20230221091203_0066_V','DJI_20230221091245_0079_V','DJI_20230221091246_0080_V','DJI_20230221091120_0051_V','DJI_20230221091146_0060_V','DJI_20230221091212_0070_V','DJI_20230221091047_0039_V','DJI_20230221091051_0041_V','DJI_20230221091059_0044_V','20230215_093545','20221130_134447','20230204_102517','20220711_082019'] {allow-input: true}

#### Yoni ####
iii = 1
image = 'DJI_20230221095512_0069_V' #@param ['IMG-20220821-WA0000','IMG-20190207-WA0024','IMG-20220913-WA0061','DJI_20230221095523_0076_V','DJI_20230221095512_0069_V','DJI_20230221100140_0114_V','DJI_20230221095222_0016_V','DJI_20230221095228_0020_V','DJI_20230221095235_0022_V','DJI_20230221095216_0013_V','DJI_20230221095050_0050_V','DJI_20230221095049_0049_V','DJI_20230221093958_0112_V','DJI_20230221091309_0089_V','DJI_20230221091316_0091_V','DJI_20230221091300_0086_V','DJI_20230221091322_0095_V','DJI_20230221091254_0083_V','DJI_20230221091256_0084_V','DJI_20230221091306_0088_V','DJI_20230221091105_0046_V','DJI_20230221091125_0053_V','DJI_20230221091056_0043_V','DJI_20230221091042_0037_V','20221211_122752','20221015_075959','20230228_125632','20230208_130836','20230228_125322','20220705_115213']{allow-input: true}

#### Arik ####
# iii = 2
# image = 'DJI_20230221100215_0124_V' #@param ['IMG-20230129-WA0008', 'DJI_20230221100215_0124_V', 'IMG-20190207-WA0030', 'IMG-20220911-WA0037', 'IMG-20230216-WA0035', 'DJI_20230221100213_0123_V', 'IMG-20190207-WA0014', 'DJI_20230221095459_0062_V', 'DJI_20230221095305_0034_V', 'DJI_20230221095615_0090_V', 'DJI_20230221095518_0073_V', 'DJI_20230221095219_0014_V', 'DJI_20230221095226_0018_V', 'DJI_20230221095251_0028_V','DJI_20230221095214_0012_V', 'DJI_20230221095213_0011_V', 'DJI_20230221095211_0010_V', 'DJI_20230221095153_0001_V', 'DJI_20230221094014_0128_V', 'DJI_20230221095025_0031_V', 'DJI_20230221091850_0113_V', 'DJI_20230221091258_0085_V', 'DJI_20230221091136_0057_V', 'DJI_20230221091131_0055_V', 'DJI_20230221081744_0017_V', 'DJI_20230221090557_0040_V','DJI_20230221091040_0036_V','DJI_20230221091035_0034_V','20230301_094628','20230228_125320','20230301_094601']{allow-input: true}

split = "6" #@param [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15] {allow-input: true}
split = int(split)

folder_path = folder_paths[iii]
worksheet = sh.get_worksheet(iii)
values = worksheet.get_all_values()
df = pd.DataFrame(values[1:], columns=values[0])
df.index = df["image\split"]
df = df.drop(columns=["image\split"]).rename_axis(None, axis=0)

show_image_and_split(image,split)

In [ ]:
#@markdown <font color='green'><font size=5>**3.  Annotate**</font>

annotate_one_sp(image,split)

In [ ]:
#@markdown <font color='green'><font size=5>**4.  Show Masks**</font>
masks = "Not Annotated" #@param ["All", "Annotated", "Not Annotated"] {allow-input: true}
show_masks_grid(image, split, masks)